# Structuring E2E Applications

### Loading OpenAI API Key

In [10]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv(dotenv_path='../../.env')  # Specify the path to your .env file

# Access the environment variable
api_key = os.getenv('OPENAI_API_KEY')

# Check if the variable is loaded
if api_key or api_key == "":
    print("API key loaded successfully.")
else:
    print("Failed to load API key.")

API key loaded successfully.


## Initializing OpenAI API

In [11]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

## Auxiliary Functions

### Function to get the model response

In [12]:
class Message:
    def __init__(self, role, content):
        self.role = role
        self.content = content

In [13]:
from tenacity import retry, wait_random_exponential, stop_after_attempt

@retry(wait=(wait_random_exponential(min=5, max=40)), stop=stop_after_attempt(4))
def get_response(req_msgs:list[Message], model="gpt-4o-mini", temperature=0.7):
    """
    Get a response from the OpenAI API.

    Parameters:
    - prompt (str): The input prompt for the model.
    - model (str): The model to use.
    - temperature (float): Sampling temperature. Default is 0.7.

    Returns:
    - str: The model's response.
    """
    model_response = client.chat.completions.create(
        model=model,
        messages=[message.__dict__ for message in req_msgs],
        temperature=temperature,
        response_format={ "type": "json_object" }
    )
    return model_response.choices[0].message.content

In [14]:
import tiktoken

class MessageLengthError(Exception):
    """Custom exception for message length errors."""
    def __init__(self, errors:[], message="Message length validation failed."):
        """
        Initialize the MessageLengthError exception.
        :param errors:
        :param message:
        """
        self.errors = errors
        super().__init__(message)

    def get_errors(self):
        """
        Get the list of errors.
        :return: List of errors.
        """
        return self.errors

def count_tokens(input_message: Message):
    """
    Count the number of tokens in a message.

    Parameters:
    - input_message (Message): The message to count tokens for.

    Returns:
    - int: The number of tokens in the message.
    """
    my_encoding = tiktoken.encoding_for_model("gpt-4o-mini")
    return len(my_encoding.encode(input_message.content))

def validate_messages(msgs_to_validate: list[Message], max_total_tokens=4096, max_tokens_per_message=2048):
    """
    Validate the messages to ensure they do not exceed the token limit.

    Parameters:
    - messages (list[Message]): The list of messages to validate.
    - max_tokens (int): The maximum number of tokens allowed. Default is 4096.
    """
    errors = []
    total_tokens = 0
    for i, message in enumerate(msgs_to_validate):
        msg_tokens = count_tokens(message)
        total_tokens += msg_tokens
        if msg_tokens > max_tokens_per_message:
            errors.append(f"Message {i+1} exceeds token limit: {msg_tokens} tokens (max {max_tokens_per_message} tokens)")

    if total_tokens > max_total_tokens:
        errors.append(f"Total tokens exceed limit: {total_tokens} tokens (max {max_total_tokens} tokens)")

    if len(errors) > 0:
        raise MessageLengthError(errors=errors, message="Message length validation failed.")



### Examples

In [15]:
books = "The Great Gatsby, To Kill a Mockingbird, 1984, Pride and Prejudice, The Catcher in the Rye"
prompt = f"For each book in {books} find the author and the year of publication, and return the results in a JSON format."
messages = [Message(role="user", content=prompt)]
response = get_response(req_msgs=messages)
print(f"Prompt:\n{prompt}")
print(f"\nResponse:\n{response}")

Prompt:
For each book in The Great Gatsby, To Kill a Mockingbird, 1984, Pride and Prejudice, The Catcher in the Rye find the author and the year of publication, and return the results in a JSON format.

Response:
{
  "books": [
    {
      "title": "The Great Gatsby",
      "author": "F. Scott Fitzgerald",
      "year_of_publication": 1925
    },
    {
      "title": "To Kill a Mockingbird",
      "author": "Harper Lee",
      "year_of_publication": 1960
    },
    {
      "title": "1984",
      "author": "George Orwell",
      "year_of_publication": 1949
    },
    {
      "title": "Pride and Prejudice",
      "author": "Jane Austen",
      "year_of_publication": 1813
    },
    {
      "title": "The Catcher in the Rye",
      "author": "J.D. Salinger",
      "year_of_publication": 1951
    }
  ]
}


In [16]:
# Validate the messages
long_prompt = "This is a very long prompt that exceeds the token limit. " * 100
long_messages = [Message(role="user", content=long_prompt)]
try:
    validate_messages(long_messages, max_total_tokens=4096, max_tokens_per_message=1000)
    print(f"Messages are valid.")
except MessageLengthError as e:
    errors = e.get_errors()
    for error in errors:
        print(f"Error: {error}")

Error: Message 1 exceeds token limit: 1201 tokens (max 1000 tokens)
